In [5]:
from bs4 import BeautifulSoup
import pandas as pd
import re
import requests
import time

**Этап 1**

In [24]:
hrefs = []

In [25]:
root = "https://shop.relod.ru"

In [26]:
post_url = "https://shop.relod.ru/bitrix/components/bxready.market2/catalog.section/ajax.php"

In [27]:
data = {
    "siteId": "s1",
    "template": ".default.5066805247fd90f4681c9458759019c47aeeee73c2035d99874d41cd9e7eb382",
    "parameters": "YToxNzM6e3M6MTg6IkNPTVBPTkVOVF9URU1QTEFURSI7czo4OiIuZGVmYXVsdCI7czoxMToiSUJMT0NLX1RZUEUiO3M6NzoiY2F0YWxvZyI7czo5OiJJQkxPQ0tfSUQiO3M6MToiMiI7czoxMDoiU0VDVElPTl9JRCI7czo0OiI0NTc3IjtzOjEyOiJTRUNUSU9OX0NPREUiO047czoxMToiRklMVEVSX05BTUUiO3M6MTY6IkNBVEFMT0dfUFJPRFVDVFMiO3M6MTk6IklOQ0xVREVfU1VCU0VDVElPTlMiO3M6MToiQSI7czoxOToiU0hPV19BTExfV09fU0VDVElPTiI7czoxOiJOIjtzOjEzOiJDVVNUT01fRklMVEVSIjtzOjA6IiI7czoxODoiSElERV9OT1RfQVZBSUxBQkxFIjtzOjE6Ik4iO3M6MjU6IkhJREVfTk9UX0FWQUlMQUJMRV9PRkZFUlMiO3M6MToiWSI7czoxODoiRUxFTUVOVF9TT1JUX0ZJRUxEIjtzOjE1OiJQUk9QRVJUWV9SQVRJTkciO3M6MTg6IkVMRU1FTlRfU09SVF9PUkRFUiI7czo0OiJkZXNjIjtzOjE5OiJFTEVNRU5UX1NPUlRfRklFTEQyIjtzOjQ6Im5hbWUiO3M6MTk6IkVMRU1FTlRfU09SVF9PUkRFUjIiO3M6NDoiZGVzYyI7czoxNzoiT0ZGRVJTX1NPUlRfRklFTEQiO3M6MjoiaWQiO3M6MTc6Ik9GRkVSU19TT1JUX09SREVSIjtzOjM6ImFzYyI7czoxODoiT0ZGRVJTX1NPUlRfRklFTEQyIjtzOjU6InNob3dzIjtzOjE4OiJPRkZFUlNfU09SVF9PUkRFUjIiO3M6MzoiYXNjIjtzOjE4OiJQQUdFX0VMRU1FTlRfQ09VTlQiO3M6MjoiMjAiO3M6MTg6IkxJTkVfRUxFTUVOVF9DT1VOVCI7czoxOiI0IjtzOjIwOiJQUk9QRVJUWV9DT0RFX01PQklMRSI7YToyOntpOjA7czo4OiJBVFJfSVNCTiI7aToxO3M6MTM6IkFUUl9QVUJMSVNIRVIiO31zOjE3OiJPRkZFUlNfRklFTERfQ09ERSI7YTo2OntpOjA7czo0OiJOQU1FIjtpOjE7czoxMjoiUFJFVklFV19URVhUIjtpOjI7czoxNToiUFJFVklFV19QSUNUVVJFIjtpOjM7czoxMToiREVUQUlMX1RFWFQiO2k6NDtzOjE0OiJERVRBSUxfUElDVFVSRSI7aTo1O3M6MDoiIjt9czoyMDoiT0ZGRVJTX1BST1BFUlRZX0NPREUiO2E6Mjp7aTowO3M6NjoiTk9USUNFIjtpOjE7czowOiIiO31zOjEyOiJPRkZFUlNfTElNSVQiO3M6MToiMCI7czoyMDoiUFJPRFVDVF9ESVNQTEFZX01PREUiO3M6MToiWSI7czoxMzoiQUREX1BJQ1RfUFJPUCI7czoxNzoiQVRSX0VYVFJBX1BJQ1RVUkUiO3M6MTk6Ik9GRkVSX0FERF9QSUNUX1BST1AiO3M6MToiLSI7czoxNjoiT0ZGRVJfVFJFRV9QUk9QUyI7YToxOntpOjA7czo2OiJOT1RJQ0UiO31zOjIwOiJQUk9EVUNUX1NVQlNDUklQVElPTiI7czoxOiJZIjtzOjIxOiJTSE9XX0RJU0NPVU5UX1BFUkNFTlQiO3M6MToiWSI7czoxNDoiU0hPV19PTERfUFJJQ0UiO3M6MToiWSI7czoxNzoiU0hPV19NQVhfUVVBTlRJVFkiO3M6MToiQSI7czoyMToiQlhSX1NIT1dfTUFYX1FVQU5USVRZIjtzOjE6IkEiO3M6MTY6IlNIT1dfQ0xPU0VfUE9QVVAiO3M6MToiWSI7czoxMjoiTUVTU19CVE5fQlVZIjtzOjEyOiLQmtGD0L/QuNGC0YwiO3M6MjI6Ik1FU1NfQlROX0FERF9UT19CQVNLRVQiO3M6MTc6ItCSINC60L7RgNC30LjQvdGDIjtzOjE4OiJNRVNTX0JUTl9TVUJTQ1JJQkUiO3M6NDQ6ItCj0LLQtdC00L7QvNC40YLRjCDQviDQv9C+0YHRgtGD0L/Qu9C10L3QuNC4IjtzOjE1OiJNRVNTX0JUTl9ERVRBSUwiO3M6MzQ6ItCf0L7QtNGA0L7QsdC90LXQtSDQviDRgtC+0LLQsNGA0LUiO3M6MTg6Ik1FU1NfTk9UX0FWQUlMQUJMRSI7czoyNDoi0J3QtdGCINCyINC90LDQu9C40YfQuNC4IjtzOjExOiJTRUNUSU9OX1VSTCI7czozMToiL2NhdGFsb2ctcHJvZHVjdHMvI1NFQ1RJT05fSUQjLyI7czoxMDoiREVUQUlMX1VSTCI7czozMzoiL2NhdGFsb2ctcHJvZHVjdHMvI0VMRU1FTlRfQ09ERSMvIjtzOjE5OiJTRUNUSU9OX0lEX1ZBUklBQkxFIjtzOjEwOiJTRUNUSU9OX0lEIjtzOjEwOiJDQUNIRV9UWVBFIjtzOjE6IkEiO3M6MTA6IkNBQ0hFX1RJTUUiO3M6ODoiMzYwMDAwMDAiO3M6MTI6IkNBQ0hFX0ZJTFRFUiI7czoxOiJZIjtzOjEyOiJDQUNIRV9HUk9VUFMiO3M6MToiWSI7czo5OiJTRVRfVElUTEUiO3M6MToiWSI7czoxMzoiQlJPV1NFUl9USVRMRSI7czoxOiItIjtzOjEzOiJNRVRBX0tFWVdPUkRTIjtzOjE6Ii0iO3M6MTY6Ik1FVEFfREVTQ1JJUFRJT04iO3M6MToiLSI7czoxNzoiU0VUX0xBU1RfTU9ESUZJRUQiO3M6MToiWSI7czoyNDoiVVNFX01BSU5fRUxFTUVOVF9TRUNUSU9OIjtzOjE6IlkiO3M6MTg6IkFERF9TRUNUSU9OU19DSEFJTiI7czoxOiJOIjtzOjE1OiJBQ1RJT05fVkFSSUFCTEUiO3M6NjoiYWN0aW9uIjtzOjE5OiJQUk9EVUNUX0lEX1ZBUklBQkxFIjtzOjI6ImlkIjtzOjEwOiJQUklDRV9DT0RFIjthOjE6e2k6MDtzOjEwOiJCQVNFX1BSSUNFIjt9czoxNToiVVNFX1BSSUNFX0NPVU5UIjtzOjE6Ik4iO3M6MTY6IlNIT1dfUFJJQ0VfQ09VTlQiO3M6MDoiIjtzOjE3OiJQUklDRV9WQVRfSU5DTFVERSI7czoxOiJZIjtzOjE2OiJDT05WRVJUX0NVUlJFTkNZIjtzOjE6IlkiO3M6MTE6IkNVUlJFTkNZX0lEIjtzOjM6IlJVQiI7czoxMDoiQkFTS0VUX1VSTCI7czoyMDoiL3BlcnNvbmFsL2Jhc2tldC5waHAiO3M6MjA6IlVTRV9QUk9EVUNUX1FVQU5USVRZIjtzOjE6IlkiO3M6MjU6IlBST0RVQ1RfUVVBTlRJVFlfVkFSSUFCTEUiO3M6ODoicXVhbnRpdHkiO3M6MjQ6IkFERF9QUk9QRVJUSUVTX1RPX0JBU0tFVCI7czoxOiJZIjtzOjIyOiJQUk9EVUNUX1BST1BTX1ZBUklBQkxFIjtzOjQ6InByb3AiO3M6MjY6IlBBUlRJQUxfUFJPRFVDVF9QUk9QRVJUSUVTIjtzOjE6IlkiO3M6MTg6IlBST0RVQ1RfUFJPUEVSVElFUyI7YTo0OntpOjA7czoxMDoiQVRSX0FVVEhPUiI7aToxO3M6MTI6IkFUUl9MQU5HVUFHRSI7aToyO3M6NzoiQVRSX0FHRSI7aTozO3M6MTA6IkFUUl9PUklHSU4iO31zOjIyOiJPRkZFUlNfQ0FSVF9QUk9QRVJUSUVTIjthOjE6e2k6MDtzOjY6Ik5PVElDRSI7fXM6MjA6IkFERF9UT19CQVNLRVRfQUNUSU9OIjtOO3M6MTU6IkRJU1BMQVlfQ09NUEFSRSI7czoxOiJOIjtzOjE0OiJQQUdFUl9URU1QTEFURSI7czo4OiIuZGVmYXVsdCI7czoxNzoiRElTUExBWV9UT1BfUEFHRVIiO3M6MToiWSI7czoyMDoiRElTUExBWV9CT1RUT01fUEFHRVIiO3M6MToiWSI7czoxMToiUEFHRVJfVElUTEUiO3M6MTI6ItCi0L7QstCw0YDRiyI7czoxNzoiUEFHRVJfU0hPV19BTFdBWVMiO3M6MToiTiI7czoyMDoiUEFHRVJfREVTQ19OVU1CRVJJTkciO3M6MToiTiI7czozMToiUEFHRVJfREVTQ19OVU1CRVJJTkdfQ0FDSEVfVElNRSI7czo1OiIzNjAwMCI7czoxNDoiUEFHRVJfU0hPV19BTEwiO3M6MToiTiI7czoyMjoiUEFHRVJfQkFTRV9MSU5LX0VOQUJMRSI7czoxOiJOIjtzOjE1OiJQQUdFUl9CQVNFX0xJTksiO047czoxNzoiUEFHRVJfUEFSQU1TX05BTUUiO047czo5OiJMQVpZX0xPQUQiO3M6MToiWSI7czoxODoiTUVTU19CVE5fTEFaWV9MT0FEIjtzOjIzOiLQn9C+0LrQsNC30LDRgtGMINC10YnRkSI7czoxNDoiTE9BRF9PTl9TQ1JPTEwiO3M6MToiTiI7czoxNDoiU0VUX1NUQVRVU180MDQiO3M6MToiWSI7czo4OiJTSE9XXzQwNCI7czoxOiJZIjtzOjg6IkZJTEVfNDA0IjtzOjg6Ii80MDQucGhwIjtzOjExOiJNRVNTQUdFXzQwNCI7czowOiIiO3M6MTU6IkNPTVBBVElCTEVfTU9ERSI7czoxOiJZIjtzOjI4OiJESVNBQkxFX0lOSVRfSlNfSU5fQ09NUE9ORU5UIjtzOjE6Ik4iO3M6MjI6IlVTRV9FTkhBTkNFRF9FQ09NTUVSQ0UiO3M6MToiWSI7czoxMjoiRU5MQVJHRV9QUk9QIjtzOjA6IiI7czoxMjoiQ09NUEFSRV9QQVRIIjtzOjUwOiIvY2F0YWxvZy1wcm9kdWN0cy9jb21wYXJlLnBocD9hY3Rpb249I0FDVElPTl9DT0RFIyI7czoxNjoiTUVTU19CVE5fQ09NUEFSRSI7czoxNjoi0KHRgNCw0LLQvdC40YLRjCI7czoxMjoiQ09NUEFSRV9OQU1FIjtzOjIwOiJDQVRBTE9HX0NPTVBBUkVfTElTVCI7czoyMjoiTUVTU19TSE9XX01BWF9RVUFOVElUWSI7czoxNDoi0J3QsNC70LjRh9C40LUiO3M6MjQ6IlJFTEFUSVZFX1FVQU5USVRZX0ZBQ1RPUiI7czoyOiIxMCI7czoyNzoiTUVTU19SRUxBVElWRV9RVUFOVElUWV9NQU5ZIjtzOjEwOiLQvNC90L7Qs9C+IjtzOjI2OiJNRVNTX1JFTEFUSVZFX1FVQU5USVRZX0ZFVyI7czo4OiLQvNCw0LvQviI7czoxNzoiUVVBTlRJVFlfSU5fU1RPQ0siO3M6MTc6ItCSINC90LDQu9C40YfQuNC4IjtzOjE4OiJRVUFOVElUWV9PVVRfU1RPQ0siO3M6MTc6ItCf0L7QtCDQt9Cw0LrQsNC3IjtzOjI0OiJISURFX1NFQ1RJT05fREVTQ1JJUFRJT04iO3M6MToiWSI7czoxMjoiVEhJU19VTklDX0lEIjtzOjk6IjFfc2VjdGlvbiI7czo2OiJSRUdJT04iO3M6MDoiIjtzOjIwOiJCWFJfQUpBWF9SRUdJT05fSU5GTyI7czowOiIiO3M6Mjg6IkJYUkVBRFlfRUxFTUVOVF9BRERDTEFTU19CSUciO3M6MDoiIjtzOjMwOiJCWFJFQURZX0VMRU1FTlRfQUREQ0xBU1NfU01BTEwiO3M6MDoiIjtzOjMzOiJCWFJFQURZX0VMRU1FTlRfQUREQ0xBU1NfU1RBTkRBUlQiO3M6MDoiIjtzOjI0OiJCWFJFQURZX0VMRU1FTlRfRFJBV19CSUciO3M6MTk6ImVjb21tZXJjZS5tMi5iaWcudjEiO3M6MjY6IkJYUkVBRFlfRUxFTUVOVF9EUkFXX1NNQUxMIjtzOjIxOiJlY29tbWVyY2UubTIuc21hbGwudjEiO3M6Mjk6IkJYUkVBRFlfRUxFTUVOVF9EUkFXX1NUQU5EQVJUIjtzOjE1OiJlY29tbWVyY2UubTIudjEiO3M6MzA6IkJYUkVBRFlfRUxFTUVOVF9FWFRfUEFSQU1TX0JJRyI7czoxMjoiYXJyRXh0UGFyYW1zIjtzOjMyOiJCWFJFQURZX0VMRU1FTlRfRVhUX1BBUkFNU19TTUFMTCI7czoxMjoiYXJyRXh0UGFyYW1zIjtzOjM1OiJCWFJFQURZX0VMRU1FTlRfRVhUX1BBUkFNU19TVEFOREFSVCI7czoxMjoiYXJyRXh0UGFyYW1zIjtzOjI4OiJCWFJFQURZX0xJU1RfTUFSS0VSX1RZUEVfQklHIjtzOjE1OiJyaWJib24udmVydGljYWwiO3M6MzA6IkJYUkVBRFlfTElTVF9NQVJLRVJfVFlQRV9TTUFMTCI7czozOiJub3QiO3M6MzM6IkJYUkVBRFlfTElTVF9NQVJLRVJfVFlQRV9TVEFOREFSVCI7czoxNToicmliYm9uLnZlcnRpY2FsIjtzOjMzOiJCWFJFQURZX0xJU1RfT1dOX01BUktFUl9VU0VfU01BTEwiO3M6MToiTiI7czozNjoiQlhSRUFEWV9MSVNUX09XTl9NQVJLRVJfVVNFX1NUQU5EQVJUIjtzOjE6Ik4iO3M6MjI6IkJYUkVBRFlfVVNFUl9UWVBFU19CSUciO3M6MToiTiI7czoyNDoiQlhSRUFEWV9VU0VSX1RZUEVTX1NNQUxMIjtzOjE6Ik4iO3M6Mjc6IkJYUkVBRFlfVVNFUl9UWVBFU19TVEFOREFSVCI7czoxOiJOIjtzOjI4OiJCWFJFQURZX1VTRV9FTEVNRU5UQ0xBU1NfQklHIjtzOjE6IlkiO3M6MzA6IkJYUkVBRFlfVVNFX0VMRU1FTlRDTEFTU19TTUFMTCI7czoxOiJZIjtzOjMzOiJCWFJFQURZX1VTRV9FTEVNRU5UQ0xBU1NfU1RBTkRBUlQiO3M6MToiWSI7czoyNjoiQlhSRUFEWV9WRVJUSUNBTF9BTElHTl9CSUciO3M6MToiWSI7czoyODoiQlhSRUFEWV9WRVJUSUNBTF9BTElHTl9TTUFMTCI7czoxOiJZIjtzOjMxOiJCWFJFQURZX1ZFUlRJQ0FMX0FMSUdOX1NUQU5EQVJUIjtzOjE6IlkiO3M6MjI6IkJYUl9JTUdfTUFYX0hFSUdIVF9CSUciO3M6MzoiMTgwIjtzOjI0OiJCWFJfSU1HX01BWF9IRUlHSFRfU01BTEwiO3M6MjoiOTAiO3M6Mjc6IkJYUl9JTUdfTUFYX0hFSUdIVF9TVEFOREFSVCI7czozOiIxODAiO3M6MjE6IkJYUl9JTUdfTUFYX1dJRFRIX0JJRyI7czozOiIxODAiO3M6MjM6IkJYUl9JTUdfTUFYX1dJRFRIX1NNQUxMIjtzOjI6IjkwIjtzOjI2OiJCWFJfSU1HX01BWF9XSURUSF9TVEFOREFSVCI7czozOiIxODAiO3M6Mjg6IkJYUl9QUk9EVUNUX0JMT0NLU19PUkRFUl9CSUciO3M6NTE6InBpY3R1cmUscmF0aW5nLG5hbWUsYXJ0aWNsZSxwcmV2aWV3dGV4dCxhY3Rpb250aW1lciI7czozMzoiQlhSX1BST0RVQ1RfQkxPQ0tTX09SREVSX1NUQU5EQVJUIjtzOjM5OiJwaWN0dXJlLG5hbWUsYWN0aW9udGltZXIsYXJ0aWNsZSxyYXRpbmciO3M6MjU6IkJYUl9TSE9XX0FDVElPTl9USU1FUl9CSUciO3M6MToiTiI7czoyNzoiQlhSX1NIT1dfQUNUSU9OX1RJTUVSX1NNQUxMIjtzOjE6Ik4iO3M6MzA6IkJYUl9TSE9XX0FDVElPTl9USU1FUl9TVEFOREFSVCI7czoxOiJOIjtzOjIwOiJCWFJfU0hPV19BUlRJQ0xFX0JJRyI7czoxOiJZIjtzOjIyOiJCWFJfU0hPV19BUlRJQ0xFX1NNQUxMIjtzOjE6IlkiO3M6MjU6IkJYUl9TSE9XX0FSVElDTEVfU1RBTkRBUlQiO3M6MToiWSI7czoyNToiQlhSX1NIT1dfUFJFVklFV19URVhUX0JJRyI7czoxOiJZIjtzOjE5OiJCWFJfU0hPV19SQVRJTkdfQklHIjtzOjM6ImF2ZyI7czoyMToiQlhSX1NIT1dfUkFUSU5HX1NNQUxMIjtzOjM6ImF2ZyI7czoyNDoiQlhSX1NIT1dfUkFUSU5HX1NUQU5EQVJUIjtzOjM6ImF2ZyI7czoxOToiQlhSX1NIT1dfU0xJREVSX0JJRyI7czoxOiJOIjtzOjIxOiJCWFJfU0hPV19TTElERVJfU01BTEwiO3M6MToiWSI7czoyNDoiQlhSX1NIT1dfU0xJREVSX1NUQU5EQVJUIjtzOjE6Ik4iO3M6Mjc6IkJYUl9TS1VfUFJPUFNfU0hPV19UWVBFX0JJRyI7czo2OiJzcXVhcmUiO3M6Mjk6IkJYUl9TS1VfUFJPUFNfU0hPV19UWVBFX1NNQUxMIjtzOjY6InNxdWFyZSI7czozMjoiQlhSX1NLVV9QUk9QU19TSE9XX1RZUEVfU1RBTkRBUlQiO3M6Njoic3F1YXJlIjtzOjIzOiJCWFJfU0xJREVSX0lOVEVSVkFMX0JJRyI7czo0OiIzMDAwIjtzOjI4OiJCWFJfU0xJREVSX0lOVEVSVkFMX1NUQU5EQVJUIjtzOjQ6IjMwMDAiO3M6Mjg6IkJYUl9TTElERVJfUFJPR1JFU1NfU1RBTkRBUlQiO3M6MToiTiI7czoyODoiQlhSX1RJTEVfU0hPV19QUk9QRVJUSUVTX0JJRyI7czoxOiJZIjtzOjMwOiJCWFJfVElMRV9TSE9XX1BST1BFUlRJRVNfU01BTEwiO3M6MToiTiI7czozMzoiQlhSX1RJTEVfU0hPV19QUk9QRVJUSUVTX1NUQU5EQVJUIjtzOjE6IlkiO3M6MjE6IkJYUl9VU0VfRkFTVF9WSUVXX0JJRyI7czoxOiJOIjtzOjIzOiJCWFJfVVNFX0ZBU1RfVklFV19TTUFMTCI7czoxOiJOIjtzOjI2OiJCWFJfVVNFX0ZBU1RfVklFV19TVEFOREFSVCI7czoxOiJOIjtzOjIyOiJNRVNTX0JUTl9GQVNUX1ZJRVdfQklHIjtzOjMxOiLQkdGL0YHRgtGA0YvQuSDQv9GA0L7RgdC80L7RgtGAIjtzOjI0OiJNRVNTX0JUTl9GQVNUX1ZJRVdfU01BTEwiO3M6MzE6ItCR0YvRgdGC0YDRi9C5INC/0YDQvtGB0LzQvtGC0YAiO3M6Mjc6Ik1FU1NfQlROX0ZBU1RfVklFV19TVEFOREFSVCI7czozMToi0JHRi9GB0YLRgNGL0Lkg0L/RgNC+0YHQvNC+0YLRgCI7czoyMDoiUFJPRFVDVF9ST1dfVkFSSUFOVFMiO3M6MTY2OiJbeydWQVJJQU5UJzonMycsJ0JJR19EQVRBJzpmYWxzZX0seydWQVJJQU5UJzonMycsJ0JJR19EQVRBJzpmYWxzZX0seydWQVJJQU5UJzonMycsJ0JJR19EQVRBJzpmYWxzZX0seydWQVJJQU5UJzonMycsJ0JJR19EQVRBJzpmYWxzZX0seydWQVJJQU5UJzonMycsJ0JJR19EQVRBJzpmYWxzZX1dIjtzOjEzOiJQUk9QRVJUWV9DT0RFIjthOjI6e2k6MDtzOjg6IkFUUl9JU0JOIjtpOjE7czoxMzoiQVRSX1BVQkxJU0hFUiI7fXM6MTc6IkNVUlJFTlRfQkFTRV9QQUdFIjtzOjY2OiIvY2F0YWxvZy1wcm9kdWN0cy80NTc3Lz92aWV3PXRpdGxlJnNvcnQ9UFJPUEVSVFlfUkFUSU5HJm9yZGVyPWRlc2MiO3M6MTE6IlBBUkVOVF9OQU1FIjtzOjIzOiJieHJlYWR5Lm1hcmtldDI6Y2F0YWxvZyI7czoyMDoiUEFSRU5UX1RFTVBMQVRFX05BTUUiO3M6MTM6InJlbG9kLWNhdGFsb2ciO3M6MjA6IlBBUkVOVF9URU1QTEFURV9QQUdFIjtzOjc6InNlY3Rpb24iO3M6MTM6IkdMT0JBTF9GSUxURVIiO2E6MDp7fX0=.b41f84069c42bfff0e8e18f2596022ec341f36965c4b377427cabc7844d17928",
    "action": "showMore",
    "PAGEN_1": 1
}

In [28]:
for i in range(250):
    html_pg = requests.post(post_url, data=data).text
    soup = BeautifulSoup(html_pg, 'lxml')
    divs = soup.find_all('div', attrs={'class':"bxr-element-name"})
    for d in divs:
        hrefs.append(d.find('a').get('href'))
    data["PAGEN_1"] += 1

In [29]:
len(hrefs)

5000

In [30]:
hrefs = list(map(lambda x: root + x, hrefs))

**Этап 2**

In [43]:
def get_page(url, n_attempts=5, t_sleep=1, **kwargs):
    for i in range(n_attempts):
        page = requests.get(url)
        if page.status_code == 200:
            return BeautifulSoup(page.text, 'lxml')
        time.sleep(t_sleep)

In [44]:
def process_page(url):
    book_info = {}
    
    book_info["url"] = url
    
    soup = get_page(url)
    if soup is None:
        return book_info
    
    book_info["Название"] = soup.find('h1').text
    
    table = soup.find_all('tr', attrs={'itemprop':"additionalProperty"})
    for i in table:
        name = i.find('td', attrs={'class':"bxr-props-name"}).text
        val = re.sub(r'\n', '', i.find('td', attrs={'class':"bxr-props-data"}).text)
        book_info[name] = val
    
    marks = list(filter(None, soup.find('div', attrs={'class':"bxr-ribbon-marker-vertical"}).text.split('\n')))
    if len(marks) > 0:
        book_info["Метки"] = marks
    
    img = soup.find('img',  attrs={'itemprop':"image"})
    if img:
        book_info["Иллюстрации"] = ['https:' + img['data-src']]
    
    rate = soup.find('meta',  attrs={'itemprop':"ratingValue"})
    if rate:
        book_info["Оценка"] = float(rate['content'])
    
    cnt = soup.find('meta',  attrs={'itemprop':"ratingCount"})
    if cnt:
        book_info["Число голосов"] = int(cnt['content'])
    
    book_info["Наличие"] = soup.find('div',  attrs={'itemprop':"availability"}).text
    
    price = soup.find('meta', attrs={'itemprop':"price"})
    if price:
        book_info["Цена"] = float(price['content'])
    
    if soup.find('div', attrs={'class':"ptv-block"}):
        scripts = str(soup.find_all('script'))
        book_info["Цена (скидка)"] = float(re.search(r'"PRICE_FORMAT": "(\d+)', scripts).group(1))
    
    book_info["Описание"] = re.sub(r'\s+', ' ',soup.find('div',  attrs={'class':"bxr-detail"}).text)
    
    return book_info

In [7]:
import sys

In [49]:
import gzip
import json
import codecs
from tqdm import tqdm

from multiprocessing.dummy import Pool, Queue

queue = Queue()
for href in hrefs:
  queue.put(href)


def process_page_wrapper(i):
    with gzip.open('data/part_{:05d}.jsonl.gz'.format(i), mode='wb') as f_json:
        f_json = codecs.getwriter('utf8')(f_json)

        while not queue.empty():
            url = queue.get()
            try:
                record = process_page(url)
                record_str = json.dumps(record, ensure_ascii=False)
                print(record_str, file=f_json)
            
            except:
                with lock:
                    print("Error log: {url}", file=sys.stderr)

            with lock:
                    pbar.update(1)


with Pool(processes=8) as pool, tqdm(total=queue.qsize()) as pbar:
    lock = pbar.get_lock()
    pool.map(process_page_wrapper, range(pool._processes))

100%|██████████| 5000/5000 [47:48<00:00,  1.74it/s]  


**Этап 3**

In [1]:
import os

In [36]:
import pandas as pd

from itertools import chain         # рекомендуется использовать
from contextlib import ExitStack    # рекомендуется использовать

from typing import Generator, Dict, Any


def records_reader(dirname: str) -> Generator[Dict[str, Any], None, None]:
    base, _, files = next(os.walk(dirname))
    files = [os.path.join(base, f) for f in files if re.match('part_\d{5}.jsonl.gz', f)]
    stack = ExitStack()
    with stack:
        files = [gzip.open(f, mode='rb') for f in files]
        files = chain.from_iterable([stack.enter_context(codecs.getreader('utf8')(f)) for f in files])
        for d in files:
            yield json.loads(d)

df = pd.DataFrame(records_reader('data'))
df.to_csv('hw_3.csv', index=False)

In [37]:
pd.read_csv('hw_3.csv')

,url,Название,ISBN,Издатель,Автор,Язык,Возрастные ограничения,Рекомендованный возраст,Формат,Страницы,...,Цена,Описание,Серия,Метки,Цена (скидка),Читательская аудитория,Носитель,Тип продукта,Издание,Обзор/Ролик
0,https://shop.relod.ru/catalog-products/the_sil...,The Silmarillion,9780261102736,Harper Collins(сайт издательства),Tolkien J.R.R.,English,16+,16+,Paperback,480.0,...,678.0,Designed to take fans of The Hobbit and The Lo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://shop.relod.ru/catalog-products/the-pil...,The Pillars of the Earth,9781509886067,Pan Macmillan(сайт издательства),Follett Ken,English,16+,16+,Paperback,1104.0,...,774.0,"A spellbinding epic tale of ambition, anarchy,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://shop.relod.ru/catalog-products/zuckerm...,Zuckerman Unbound,9780099477563,Vintage(сайт издательства),Roth Philip,English,16+,16+,Paperback,240.0,...,871.0,"Following the wild success of his novel, Carno...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://shop.relod.ru/catalog-products/zombie/,Zombie,9780749953379,Piatkus(сайт издательства),Golden Christopher,English,16+,16+,Paperback,512.0,...,1113.0,RESURRECTION! The hungry dead have risen. They...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://shop.relod.ru/catalog-products/zennor_...,Zennor in Darkness,9780141033600,Penguin(сайт издательства),Dunmore Helen,English,16+,16+,Paperback,320.0,...,871.0,"In her prize-winning first novel, Zennor in Da...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,https://shop.relod.ru/catalog-products/the_acc...,The Accidental Bride,9780352347626,Black Lace,Portia Da Costa,English,16+,16+,Paperback,368.0,...,989.0,Marrying a billionaire? It's every girl's fant...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4996,https://shop.relod.ru/catalog-products/the-abc...,The ABC Murders (Poirot),9780008308209,Harper Collins(сайт издательства),Christie Agatha,English,16+,16+,Paperback,272.0,...,763.0,John Malkovich stars as Hercule Poirot in Agat...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4997,https://shop.relod.ru/catalog-products/the_120...,The 120 Days of Sodom,9780141394343,Penguin(сайт издательства),Marquis de Sade,English,16+,16+,Paperback,464.0,...,1258.0,WINNER OF THE 2017 SCOTT MONCRIEFF PRIZE A new...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4998,https://shop.relod.ru/catalog-products/that_lo...,That Loving Feeling,9780751551501,Sphere(сайт издательства),Matthews Carole,English,16+,16+,Paperback,448.0,...,1113.0,Juliet Joyce has been happily married to Rick ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
